In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('../../../Data/DL_Tutorial/Boltzmann Machines/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
print(movies.head())
print()
print(movies.info())
print()
print(movies.describe())

   0                                   1                             2
0  1                    Toy Story (1995)   Animation|Children's|Comedy
1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
2  3             Grumpier Old Men (1995)                Comedy|Romance
3  4            Waiting to Exhale (1995)                  Comedy|Drama
4  5  Father of the Bride Part II (1995)                        Comedy

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3883 non-null   int64 
 1   1       3883 non-null   object
 2   2       3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB
None

                 0
count  3883.000000
mean   1986.049446
std    1146.778349
min       1.000000
25%     982.500000
50%    2010.000000
75%    2980.500000
max    3952.000000


In [3]:
users = pd.read_csv('../../../Data/DL_Tutorial/Boltzmann Machines/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
print(users.head())
print()
print(users.info())
print()
print(users.describe())

   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117
3  4  M  45   7  02460
4  5  M  25  20  55455

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6040 non-null   int64 
 1   1       6040 non-null   object
 2   2       6040 non-null   int64 
 3   3       6040 non-null   int64 
 4   4       6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB
None

                 0            2            3
count  6040.000000  6040.000000  6040.000000
mean   3020.500000    30.639238     8.146854
std    1743.742145    12.895962     6.329511
min       1.000000     1.000000     0.000000
25%    1510.750000    25.000000     3.000000
50%    3020.500000    25.000000     7.000000
75%    4530.250000    35.000000    14.000000
max    6040.000000    56.000000    20.000000


In [4]:
ratings = pd.read_csv('../../../Data/DL_Tutorial/Boltzmann Machines/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
print(ratings.head())
print()
print(ratings.info())
print()
print(users.describe())

   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   0       1000209 non-null  int64
 1   1       1000209 non-null  int64
 2   2       1000209 non-null  int64
 3   3       1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB
None

                 0            2            3
count  6040.000000  6040.000000  6040.000000
mean   3020.500000    30.639238     8.146854
std    1743.742145    12.895962     6.329511
min       1.000000     1.000000     0.000000
25%    1510.750000    25.000000     3.000000
50%    3020.500000    25.000000     7.000000
75%    4530.250000    35.000000    14.000000
max    6040.000000    56.000000    20.000000


In [5]:
training_set = pd.read_csv('../../../Data/DL_Tutorial/Boltzmann Machines/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('../../../Data/DL_Tutorial/Boltzmann Machines/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')
print(training_set.shape)
print(test_set.shape)

(79999, 4)
(19999, 4)


In [6]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print(nb_users, nb_movies)

943 1682


In [7]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)


In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [9]:
class SAE(nn.Module): # Stacked Auto Encoder
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) # First Fully connection Encoding
        self.fc2 = nn.Linear(20, 10) # Second Fully Connection Encoding
        self.fc3 = nn.Linear(10, 20) # Third Fully Connection Encoding to Decoding
        self.fc4 = nn.Linear(20, nb_movies) # Decoding Output Vector 과 Input Vector가 같아야 함
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=5e-4)

#### Training the SAE (Stacked Auto Encoder)

In [11]:
nb_epochs = 200
for epoch in range(1, nb_epochs+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward() # optimizer 방향 결정
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step() # optimizer 강도 결정
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))


epoch: 1 loss: 1.5653344900061672
epoch: 2 loss: 1.7417073709741806
epoch: 3 loss: 1.8239275623839761
epoch: 4 loss: 1.8521137599537636
epoch: 5 loss: 1.83865061617627
epoch: 6 loss: 1.8941801593309386
epoch: 7 loss: 1.8789009274975723
epoch: 8 loss: 1.891277780650693
epoch: 9 loss: 1.9024912970170822
epoch: 10 loss: 1.9297562536685988
epoch: 11 loss: 1.9357161110383507
epoch: 12 loss: 1.9400203983879363
epoch: 13 loss: 1.9341524128253693
epoch: 14 loss: 1.9582714053386572
epoch: 15 loss: 1.9300445979649967
epoch: 16 loss: 1.956123640506276
epoch: 17 loss: 1.9614115933487406
epoch: 18 loss: 1.9758442479333755
epoch: 19 loss: 1.9726604123908764
epoch: 20 loss: 1.9831210093912317
epoch: 21 loss: 1.9678687428928607
epoch: 22 loss: 1.9649484131873978
epoch: 23 loss: 1.967426922758792
epoch: 24 loss: 1.9974370728498485
epoch: 25 loss: 1.9801552196876653
epoch: 26 loss: 1.9891319974741477
epoch: 27 loss: 1.9830760209900424
epoch: 28 loss: 2.0045335218144538
epoch: 29 loss: 1.9770231563810108

In [13]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.requires_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1.
print('test loss: ' + str(test_loss/s))

test loss: 1.803284737428979
